In [1]:
import os
import sys

In [2]:
os.environ["SPARK_HOME"]="/home/cloudera/Downloads/spark-2.4.4-bin-hadoop2.6"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"

# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
#os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
#os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import desc

In [4]:
sc = SparkContext()
# we initiate the StreamingContext with 10 second batch interval. #next we initiate our sqlcontext
ssc = StreamingContext(sc, 10)
ssc.checkpoint("checkpoint")

In [5]:
socket_stream = ssc.socketTextStream("localhost", 9999)

In [6]:
lines = socket_stream.window(60)

In [7]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

In [8]:
words = lines.flatMap(lambda line: line.split(" "))

In [9]:
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))

In [10]:
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)

In [11]:
def sendRecord(tup):
    word   = tup[0]
    amount = tup[1]
    print(word +" "+count)

In [12]:
tags_totals.foreachRDD(lambda rdd: rdd.foreach(sendRecord))


In [13]:
tags_totals.pprint()

In [14]:
ssc.start()

-------------------------------------------
Time: 2020-08-17 19:31:50
-------------------------------------------

-------------------------------------------
Time: 2020-08-17 19:32:00
-------------------------------------------
('#12YearsOfViratKohli', 1)
("#Alabama's", 1)
('#バルセロナ、わずか半年での', 1)
('#クーマン', 1)
('#11', 1)
('#セティエン', 1)
('#UCL', 1)
('#MondayNightFootball', 2)



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 47076)
Traceback (most recent call last):
  File "/home/cloudera/anaconda3/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/cloudera/anaconda3/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/cloudera/anaconda3/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/cloudera/anaconda3/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/cloudera/Downloads/spark-2.4.4-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/home/cloudera/Downloads/spark-2.4.4-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  Fi